<a href="https://colab.research.google.com/github/sahilnain/colab_LLM/blob/main/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb)

# RAG with LLaMa 13B

In this notebook we'll explore how we can use the open source **Llama-13b-chat** model in both Hugging Face transformers and LangChain.
At the time of writing, you must first request access to Llama 2 models via [this form](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) (access is typically granted within a few hours). If you need guidance on getting access please refer to the beginning of this [article](https://www.pinecone.io/learn/llama-2/) or [video](https://youtu.be/6iHVJyX2e50?t=175).

---

🚨 _Note that running this on CPU is sloooow. If running on Google Colab you can avoid this by going to **Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**. This should be included within the free tier of Colab._

---

We start by doing a `pip install` of all required libraries.

In [18]:
# !pip install transformers torch
!hf auth login # Might show error but the token still grants access.


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/hf", line 10, in <module>
   

# Llama pre trained model

In [17]:
from transformers import pipeline
import torch
from google.colab import userdata
userdata.get('colab_token')

model_id = "meta-llama/Llama-3.2-3B"

pipe = pipeline(
            "text-generation",
            model=model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

pipe("The key to life is")

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "The key to life is to live it, to taste experience to the utmost, to reach out eagerly and without fear for newer and richer experience. ~ Eleanor Roosevelt\nEleanor Roosevelt: Quotes\nEleanor Roosevelt (October 11, 1884 – November 7, 1962) was an American politician, diplomat, and activist. She was the longest-serving First Lady of the United States, holding the post from March 1933 until her death in 1962. She was a central figure in her husband's administration in the 1930s and 1940s, and she used her position as First Lady to advocate for social causes, particularly the rights of women and civil rights.\nI have never liked being told I was a woman, and it was always a relief to be told that I was a human being.\nI think the most important thing for a person to do in the world is to try to help other people, to be kind, and to be honest.\nI think it is better to be happy than dignified, better to be lazy than snobbish, better to be humorous than state.\nA man's 

In [ ]:
pipe("The first thought")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
import os, time, datetime, math
import jax, jax.numpy as jnp
from jax.profiler import start_trace, stop_trace, StepTraceAnnotation

from google.colab import drive
drive.mount('/content/drive')

print("JAX:", jax.__version__)
print("Devices:", jax.devices())

# options = jax.profiler.ProfileOptions()
# options.python_tracer_level = 3 # Python function call tracing
# options.host_tracer_level   = 3 # whether to record accelerator device activity
# options.device_tracer_level = 3 # how much host (CPU) activity to record

# (Optional) set before first jit/compile to dump HLOs for deeper inspection
# os.environ["XLA_FLAGS"] = "--xla_tpu_enable_tracing=true --xla_gpu_tracer_level=2 --xla_dump_to=/content/drive/MyDrive/jax_profile/xla --xla_dump_hlo_as_text"
# JAX_DISABLE_JIT=1

In [ ]:
start_trace(logdir)
# Label a region in the trace for clarity
with StepTraceAnnotation("matmul_bench"):
    pipe("The first thought")
jax.profiler.save_device_memory_profile(logdir + "/memory.prof")
stop_trace()

In [ ]:
%tensorboard --logdir logdir